In [4]:
import geopandas
import pandas as pd
import numpy as np
from shapely.geometry import Point
import time
import random

In [5]:
county_shp = geopandas.GeoDataFrame.from_file('/home/haoying/data/get_shp/县级行政区/县级行政区.shp')
county_shp['ID'] = [i for i in range(county_shp.shape[0])]
county_shp.head()

,adcode,name,childrenNu,level,parent,subFeature,geometry,ID
0,110101,东城区,0,district,"{""adcode"": 110000}",NaN,"POLYGON ((116.44364 39.87285, 116.44351 39.872...",0
1,110102,西城区,0,district,"{""adcode"": 110000}",NaN,"POLYGON ((116.32581 39.89679, 116.32581 39.897...",1
2,110105,朝阳区,0,district,"{""adcode"": 110000}",NaN,"MULTIPOLYGON (((116.59555 40.01752, 116.59820 ...",2
3,110106,丰台区,0,district,"{""adcode"": 110000}",NaN,"POLYGON ((116.25909 39.89667, 116.26005 39.896...",3
4,110107,石景山区,0,district,"{""adcode"": 110000}",NaN,"POLYGON ((116.16703 39.88876, 116.16636 39.889...",4


In [6]:
prov_shp = geopandas.GeoDataFrame.from_file('/home/haoying/data/get_shp/省级行政区/省级行政区.shp')
prov_shp.head()

,adcode,name,childrenNu,level,parent,subFeature,adchar,geometry
0,110000,北京市,16.0,province,"{""adcode"": 100000}",0.0,None,"POLYGON ((117.34861 40.58114, 117.34861 40.581..."
1,120000,天津市,16.0,province,"{""adcode"": 100000}",1.0,None,"MULTIPOLYGON (((117.76560 39.40053, 117.69970 ..."
2,130000,河北省,11.0,province,"{""adcode"": 100000}",2.0,None,"MULTIPOLYGON (((117.46749 40.64974, 117.46749 ..."
3,140000,山西省,11.0,province,"{""adcode"": 100000}",3.0,None,"POLYGON ((110.37926 34.60061, 110.29549 34.610..."
4,150000,内蒙古自治区,12.0,province,"{""adcode"": 100000}",4.0,None,"POLYGON ((97.17290 42.79526, 97.28254 42.78208..."


In [8]:
county_shp['parent'] = county_shp['parent'].apply(lambda x:x.split(": ")[1].strip("}")[:2]+'0000')

In [9]:
# prov_id -> ID
prov_id2ID = {}
for prov_id in prov_shp['adcode']:
    prov_id2ID[prov_id] = county_shp[county_shp['parent'] == prov_id].ID.tolist()

In [23]:
# 县
geom = county_shp.geometry
county_centroids = geom.centroid  # 质心
county_total_bounds = geom.total_bounds  # 总边界
# 省
prov_geom = prov_shp.geometry

/home/haoying/anaconda3/envs/torch/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# 是否处于四方边界内
def within_bound(lo, lat, bound):
    if lo < bound[0] or lo > bound[2] or lat < bound[1] or lat > bound[3]:
        return False
    else:
        return True
    

In [12]:
df = pd.read_csv('/home/haoying/res_zl15_effnet_b0/nightlights.csv')
df = df.drop('nightlights',axis=1)
df = df[['cluster_lon','cluster_lat','name']]
df = df.values

In [37]:
from shapely.validation import make_valid

prov_geom = prov_geom.apply(lambda x:make_valid(x))
geom = geom.apply(lambda x:make_valid(x))

In [39]:
match_PAC_lst = []

start = time.time()
for i, center in enumerate(df):
    if (i+1)%10000 == 0:
        print('===', i+1, '===')
        print('accumulated time:', time.time()-start)
    # 当前点
    lo = center[0]; lat = center[1]; name = center[2]
    point = Point(lo, lat)
#     print(point)
        
     # 搜索匹配省份
    prov_ids = prov_shp[prov_geom.intersects(point)]['adcode'].tolist()
#     print(prov_ids)
    match = False
    # 匹配省份
    for prov_id in prov_ids:
        ID_lst = prov_id2ID[prov_id]
#         print(ID_lst)
        # 省份内县级单元
        for idx in ID_lst:
            poly = geom[idx]
            if poly.intersects(point):
                match = True
                match_PAC_lst.append(county_shp.adcode[idx])
                break

        if match:
            break

    if not match:
        match_PAC_lst.append(None)

In [44]:
dff = pd.DataFrame(df)
dff.columns = ['cluster_lon','cluster_lat','name']
dff['PAC'] = match_PAC_lst
dff

,cluster_lon,cluster_lat,name,PAC
0,117.103271,49.553726,49.55372551347579_117.103271484375.png,150727.0
1,99.327393,31.16581,31.165809587861954_99.327392578125.png,513331.0
2,82.232666,44.73893,44.738929943073664_82.232666015625.png,652701.0
3,101.975098,27.469287,27.469287473692017_101.97509765625.png,513424.0
4,118.355713,49.181703,49.18170338770662_118.355712890625.png,150726.0
...,...,...,...,...
379163,122.76123,43.413029,43.41302868475145_122.76123046875.png,150522.0
379164,91.82373,29.75484,29.754839972510933_91.82373046875.png,540127.0
379165,120.432129,31.147006,31.14700630855656_120.43212890625.png,320506.0
379166,106.380615,42.171546,42.17154633452752_106.380615234375.png,NaN


In [45]:
dff.to_csv('/home/haoying/res_zl15_effnet_b0/center2pac_zl15.csv')